<a href="https://colab.research.google.com/github/NIKKISAHA/machineLearning/blob/main/energy_consumption_witha_customsimple_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [2]:
# Create a date range
date_range = pd.date_range(start='1/1/2022', end='12/31/2022', freq='H')

# Create a random energy consumption series
np.random.seed(0)
energy_consumption = np.random.uniform(100, 500, size=len(date_range))

# Create a DataFrame
df = pd.DataFrame({'Date': date_range, 'Energy_Consumption': energy_consumption})

# Save the DataFrame to a CSV file
df.to_csv('energy_consumption.csv', index=False)


<ipython-input-2-928fe6b10317>:2: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  date_range = pd.date_range(start='1/1/2022', end='12/31/2022', freq='H')


In [3]:
df = pd.read_csv('energy_consumption.csv')
print(df)

                     Date  Energy_Consumption
0     2022-01-01 00:00:00          319.525402
1     2022-01-01 01:00:00          386.075747
2     2022-01-01 02:00:00          341.105350
3     2022-01-01 03:00:00          317.953273
4     2022-01-01 04:00:00          269.461920
...                   ...                 ...
8732  2022-12-30 20:00:00          394.387388
8733  2022-12-30 21:00:00          139.259174
8734  2022-12-30 22:00:00          423.577350
8735  2022-12-30 23:00:00          419.397614
8736  2022-12-31 00:00:00          186.290881

[8737 rows x 2 columns]


In [4]:
data = df['Energy_Consumption'].values

# Scale data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.reshape(-1, 1))

In [5]:
# Split data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data, test_data = scaled_data[0:train_size], scaled_data[train_size:len(scaled_data)]

# Create training and testing datasets
def create_dataset(dataset, time_step=1):
    dataX, dataY = [], []
    for i in range(len(dataset)-time_step-1):
        a = dataset[i:(i+time_step), 0]
        dataX.append(a)
        dataY.append(dataset[(i+time_step), 0])
    return np.array(dataX), np.array(dataY)

time_step = 100
X_train, y_train = create_dataset(train_data, time_step)
X_test, y_test = create_dataset(test_data, time_step)

# Reshape data for LSTM model
X_train = X_train.reshape(X_train.shape[0],X_train.shape[1],1)
X_test = X_test.reshape(X_test.shape[0],X_test.shape[1],1)

In [6]:
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(units=50, return_sequences=False))
model.add(Dense(units=25))
model.add(Dense(units=1))

# Compile model
model.compile(optimizer='adam', loss='mean_squared_error')

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=2)

Epoch 1/10
216/216 - 11s - 49ms/step - loss: 0.0838
Epoch 2/10
216/216 - 20s - 95ms/step - loss: 0.0839
Epoch 3/10
216/216 - 21s - 96ms/step - loss: 0.0837
Epoch 4/10
216/216 - 11s - 49ms/step - loss: 0.0842
Epoch 5/10
216/216 - 20s - 94ms/step - loss: 0.0838
Epoch 6/10
216/216 - 20s - 94ms/step - loss: 0.0838
Epoch 7/10
216/216 - 21s - 96ms/step - loss: 0.0838
Epoch 8/10
216/216 - 20s - 94ms/step - loss: 0.0840
Epoch 9/10
216/216 - 22s - 100ms/step - loss: 0.0839
Epoch 10/10
216/216 - 20s - 91ms/step - loss: 0.0838


In [12]:
predictions = model.predict(X_test)

# Evaluate model
mse = model.evaluate(X_test, y_test)
print('MSE: ', mse)

52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step
52/52 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 0.0863
MSE:  0.08410238474607468
